# How to match? 

## 0. Import libraries 

In [21]:
import pandas as pd
import warnings
import numpy as np
from lightautoml.addons.hypex import Matcher

warnings.filterwarnings('ignore')
%config Completer.use_jedi = False

## 1. Create or upload your dataset  
In this case we will create random dataset with known effect size  
If you have your own dataset, go to the part 2 


In [22]:
# Simulating dataset with known effect size
num_users = 10000
num_months = 12

signup_months = np.random.choice(np.arange(1, num_months), num_users) * np.random.randint(0,2, size=num_users) # signup_months == 0 means customer did not sign up
df = pd.DataFrame({
    'user_id': np.repeat(np.arange(num_users), num_months),
    'signup_month': np.repeat(signup_months, num_months), # signup month == 0 means customer did not sign up
    'month': np.tile(np.arange(1, num_months+1), num_users), # months are from 1 to 12
    'spend': np.random.poisson(500, num_users*num_months) #np.random.beta(a=2, b=5, size=num_users * num_months)*1000 # centered at 500
})
# A customer is in the treatment group if and only if they signed up
df["treat"] = df["signup_month"]>0
# Simulating an effect of month (monotonically decreasing--customers buy less later in the year)
df["spend"] = df["spend"] - df["month"]*10
# Simulating a simple treatment effect of 100
after_signup = (df["signup_month"] < df["month"]) & (df["treat"])
df.loc[after_signup,"spend"] = df[after_signup]["spend"] + 100
df.head()

,user_id,signup_month,month,spend,treat
0,0,1,1,458,True
1,0,1,2,565,True
2,0,1,3,530,True
3,0,1,4,623,True
4,0,1,5,553,True


In [23]:
# Setting the signup month (for ease of analysis)
i = 3
df_i_signupmonth = (
    df[df.signup_month.isin([0, i])]
    .groupby(["user_id", "signup_month", "treat"])
    .apply(
        lambda x: pd.Series(
            {
                "pre_spends": x.loc[x.month < i, "spend"].mean(),
                "post_spends": x.loc[x.month > i, "spend"].mean(),
            }
        )
    )
    .reset_index()
)
df_i_signupmonth

,user_id,signup_month,treat,pre_spends,post_spends
0,1,0,False,497.5,418.000000
1,2,0,False,506.5,411.111111
2,4,0,False,484.5,401.444444
3,6,0,False,470.0,415.000000
4,7,0,False,478.0,398.777778
...,...,...,...,...,...
5419,9990,0,False,469.5,420.888889
5420,9991,0,False,458.5,408.111111
5421,9995,0,False,480.0,419.222222
5422,9998,0,False,466.0,415.555556


In [24]:
# Additional category features
gender = np.random.choice(a=[0,1], size=df_i_signupmonth.user_id.nunique())
age = np.random.choice(a=range(18, 70), size=df_i_signupmonth.user_id.nunique())
industry = np.random.choice(a=range(1, 3), size=df_i_signupmonth.user_id.nunique())
df_i_signupmonth['age'] = age
df_i_signupmonth['is_male'] =  gender
df_i_signupmonth['industry'] =  industry
df_i_signupmonth['industry'] = df_i_signupmonth['industry'].astype('str')
df_i_signupmonth['treat'] = df_i_signupmonth['treat'].astype(int)
df_i_signupmonth.head()

,user_id,signup_month,treat,pre_spends,post_spends,age,is_male,industry
0,1,0,0,497.5,418.000000,35,0,1
1,2,0,0,506.5,411.111111,36,1,1
2,4,0,0,484.5,401.444444,19,1,2
3,6,0,0,470.0,415.000000,57,0,2
4,7,0,0,478.0,398.777778,69,1,1


In [25]:
df_i_signupmonth.columns

Index(['user_id', 'signup_month', 'treat', 'pre_spends', 'post_spends', 'age',
       'is_male', 'industry'],
      dtype='object')

## 2. Matching  
### 2.0 Init params
info_col used to define informative attributes that should not be part of matching, such as user_id  
But to explicitly store this column in the table, so that you can compare directly after computation

In [26]:
info_col = ['user_id', 'signup_month']

outcome = 'post_spends'
treatment = 'treat'

### 2.1 Simple matching
This is the easiest way to initialize and calculate metrics on a Matching task  
Use it when you are clear about each attribute or if you don't have any additional task conditions (Strict equality for certain features) 

In [27]:
# Standard model with base parameters
model = Matcher(input_data=df_i_signupmonth, outcome=outcome, treatment=treatment, info_col=info_col)
results, quality_results, df_matched = model.estimate()

  0%|          | 0/5424 [00:00<?, ?it/s]

### 2.2 Matching with a fixed variable  
Used when you have categorical feature(s) that you want to compare by strict equality  
group_col is used for strict comparison of categorical features.  
In our case there is only one attribute  
If there are several such attributes, you should make one of them and use it

In [28]:
# group_col is used for strict comparison of categorical features.  
# In our case there is only one attribute  
# If there are several such attributes, you should make one of them and use it

group_col = "industry"

In [29]:
model = Matcher(input_data=df_i_signupmonth, outcome=outcome, treatment=treatment,
                info_col=info_col, group_col=group_col)
results, quality_results, df_matched = model.estimate()

  0%|          | 0/4 [00:00<?, ?it/s]

### 2.3 Matching but you don't know which features are better to use  
This is the easiest way to initialize and calculate metrics on a Matching task  
Use this method when you want to select the most important features and make matching based on them. 

In [30]:
model = Matcher(input_data=df_i_signupmonth, outcome=outcome, treatment=treatment,
                info_col=info_col, group_col=group_col)

In [31]:
features_importance = model.lama_feature_select()
features_importance

,Feature,Importance
0,pre_spends,23054.999756
3,industry,23054.999756
1,age,22608.830078
2,is_male,2913.379883


In [32]:
features = features_importance['Feature'].to_list()

In [33]:
# You can use both variant that you like. So you can just list the features to be matched

#results, quality_results = model.estimate(features=features_importance[:3])
results, quality_results, df_matched = model.estimate(features=features[:3])

  0%|          | 0/4 [00:00<?, ?it/s]

In [34]:
df_matched

,index,pre_spends,age,is_male,industry,pre_spends_matched,age_matched,is_male_matched,index_matched,industry_matched,post_spends,post_spends_matched,post_spends_matched_bias,treat,treat_matched
0,4990,492.5,25,1,1,491.5,25.0,1.0,[1562],1,510.777778,425.111111,85.678375,1,0
1,4991,490.5,45,1,1,490.0,45.0,1.0,[1747],1,514.222222,415.000000,99.228076,1,0
2,4992,485.5,67,0,1,485.5,67.0,1.0,[2163],1,520.666667,427.555556,93.111111,1,0
3,4993,492.5,47,0,1,493.0,47.0,1.0,[971],1,519.666667,407.111111,112.549701,1,0
4,4994,496.0,61,0,1,497.0,61.0,1.0,"[1317, 2406]",1,528.111111,426.000000,102.099403,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4985,4985,475.5,24,0,2,478.5,24.0,1.0,[251],2,415.888889,530.555556,114.741844,0,1
4986,4986,469.5,19,0,2,469.0,21.0,1.0,[341],2,420.888889,519.000000,98.043021,0,1
4987,4987,480.0,29,1,2,479.0,29.0,1.0,[252],2,419.222222,521.000000,101.752719,0,1
4988,4988,466.0,20,1,2,467.0,22.0,1.0,[307],2,415.555556,520.555556,104.969499,0,1


## 3. Results  
### 3.1 ATE, ATT, ATC

In [35]:
# model.matcher.results
results

,effect_size,std_err,p-val,ci_lower,ci_upper
ATE,100.299458,0.605344,0.0,99.112985,101.485932
ATC,100.233913,0.617081,0.0,99.024434,101.443392
ATT,101.053079,0.712787,0.0,99.656016,102.450141


### 3.2 SMD, PSI, KS-test, repeats

In [36]:
# matching quality result - SMD
model.quality_result

{'psi':         column_treated  anomaly_score_treated check_result_treated  \
 0          age_treated                   0.00                   OK   
 1     industry_treated                   0.00                   OK   
 2  post_spends_treated                  16.11                  NOK   
 3   pre_spends_treated                   0.01                   OK   
 
         column_untreated  anomaly_score_untreated check_result_untreated  
 0          age_untreated                     0.01                     OK  
 1     industry_untreated                     0.00                     OK  
 2  post_spends_untreated                     8.25                    NOK  
 3   pre_spends_untreated                     0.00                     OK  ,
 'ks_test':             match_control_to_treat  match_treat_to_control
 age                       1.000000                0.949318
 is_male                   0.028176                0.676574
 pre_spends                1.000000                0.405462,
 's

In [37]:
# matching quality result - PSI
model.quality_result['psi']

,column_treated,anomaly_score_treated,check_result_treated,column_untreated,anomaly_score_untreated,check_result_untreated
0,age_treated,0.00,OK,age_untreated,0.01,OK
1,industry_treated,0.00,OK,industry_untreated,0.00,OK
2,post_spends_treated,16.11,NOK,post_spends_untreated,8.25,NOK
3,pre_spends_treated,0.01,OK,pre_spends_untreated,0.00,OK


In [38]:
# matching quality result - KS-test

model.quality_result['ks_test']

,match_control_to_treat,match_treat_to_control
age,1.000000,0.949318
is_male,0.028176,0.676574
pre_spends,1.000000,0.405462


In [39]:
# matching quality result - repeats
model.quality_result['repeats']

{'match_control_to_treat': 0.92, 'match_treat_to_control': 0.09}

### 3.3 Validation
validate result with placebo treatment  
our new effect size is close to zero (it should be)

In [40]:
 
model.validate_result()
 

  0%|          | 0/10 [00:00<?, ?it/s]

{'post_spends': [100.23020295532162, 0.35204903782360664]}